In [1]:
#JSP.ipynb

#讀檔
import pandas as pd
import numpy as np
import time
import pulp  

processing_time=pd.read_excel("Jobshop.xlsx",sheet_name="processing_time",header = None)
machine_order=pd.read_excel("Jobshop.xlsx",sheet_name="machine_order",header = None)

pt = processing_time.values
mo = machine_order.values

model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)

# 決策變數
dv = pulp.LpVariable.dicts("start_time",((i, j) for i in range(6) for j in range(6)),lowBound=0,cat='Continuous')

b =  pulp.LpVariable.dicts("binary_var",((i, j,o) for i in range(6) for j in range(6) for o in range(6) if j<o),lowBound=0,cat='Binary')

Cmax = pulp.LpVariable('Cmax',lowBound = 0, cat='Continuous')

# 目標式 : Min Cmax
model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)
model += Cmax

# 限制式:
for j in range(6):
    for i in range(5):
       I = mo[j,i]
       K = mo[j,i+1]
       J = j
       model += (dv[K,J] - dv[I,J]) >= pt[J][list(mo[J]).index(I)]
       
for j in range(6):
    for i in range(6):
       I = mo[j,i]
       J = j
       model += (Cmax - dv[I,J]) >= pt[J][list(mo[J]).index(I)]
       
for i in range(6):
    for j in range(6):
        for l in range(6):
            if j<l:
                I=i
                L = l
                J = j
                model += (dv[I,J] - dv[I,L]) >= (pt[L][list(mo[L]).index(I)] - 100*(1-b[i,J,L]))
                model += (dv[I,L] - dv[I,J]) >= (pt[J][list(mo[J]).index(I)] - 100*(b[i,J,L]))

# 求解       
start_time = time.time()
model.solve()
pulp.LpStatus[model.status]     

for var in dv:
    var_value = dv[var].varValue
    print( var[0],'-',var[1] ,var_value)

total_cost = pulp.value(model.objective)
print ('min cost:',total_cost)
print('the elapsed time:%s'% (time.time() - start_time))

'''--------plot gantt chart-------'''
import plotly.express as px
import datetime

m_keys=[j for j in range(6)]
j_keys=[j for j in range(6)]
key_count={key:0 for key in j_keys}
j_count={key:0 for key in j_keys}
m_count={key:0 for key in m_keys}
j_record={}

for var in dv:
    var_value = dv[var].varValue
    
    start_time=str(datetime.timedelta(seconds=var_value)) # convert seconds to hours, minutes and seconds
    end_time=str(datetime.timedelta(seconds=var_value+pt[var[1]][list(mo[var[1]]).index(var[0])]))
        
    j_record[(var[1],var[0])]=[start_time,end_time]
    
    key_count[i]=key_count[i]+1
        

df=[]
for m in m_keys:
    for j in j_keys:
        df.append(dict(Task='M %s'%(m+1), Start='2020-10-15 %s'%(str(j_record[(j,m)][0])), Finish='2020-10-15 %s'%(str(j_record[(j,m)][1])),Resource='Job %s'%(j)))
    
fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Task",text="Task")
fig.show()

0 - 0 6.0
0 - 1 40.0
0 - 2 21.0
0 - 3 16.0
0 - 4 50.0
0 - 5 30.0
1 - 0 25.0
1 - 1 0.0
1 - 2 31.0
1 - 3 11.0
1 - 4 22.0
1 - 5 8.0
2 - 0 5.0
2 - 1 8.0
2 - 2 0.0
2 - 3 22.0
2 - 4 13.0
2 - 5 49.0
3 - 0 35.0
3 - 1 50.0
3 - 2 7.0
3 - 3 27.0
3 - 4 54.0
3 - 5 11.0
4 - 0 49.0
4 - 1 13.0
4 - 2 38.0
4 - 3 30.0
4 - 4 25.0
4 - 5 45.0
5 - 0 42.0
5 - 1 28.0
5 - 2 11.0
5 - 3 45.0
5 - 4 38.0
5 - 5 19.0
min cost: 55.0
the elapsed time:1.2019970417022705
